# How Does a Bike-Share Navigate Speedy Success?
### Google Data Analytics Capstone Project

# Introduction
This is my first data analytics project based on a case study given by Google Data Analytics Professional Certificate course. In this project, I will use data analytics thinking process and tools to solve this fictional real-world problem. In this report, I included following deliverables:
1. A clear statement of the business task
2. A description of all data sources used
3. Documentation of any cleaning or manipulation of data
4. A summary of analysis
5. Supporting visualizations and key findings
6. Top three recommendations based on the analysis


# Scenario
Firstly, lets back you up by the scenario of this project. In this project, I am a junior data analyst working in the marketing analyst team at Cyclistic, a bike-share company in Chicago. The director of marketing believes the company’s future success depends on maximizing the number of annual memberships. Therefore, my team wants to understand how casual riders and annual members use Cyclistic bikes differently. From these insights,
my team will design a new marketing strategy to convert casual riders into annual members. But first, Cyclistic executives must approve your recommendations, so they must be backed up with compelling data insights and professional data visualizations.

# Business Task
From the scenario, lets define our business task (i.e. the goal of this anaysis). We know that the company believes that the income from annual memberships is much better than casual user. Thus, they want to increase the number of annual memberships using our anaysis insights. So we should deliver some insights that describe each types of user behavior and trend. Hence, the question I would ask for this goal is
> What are the charateristics of annual members and casual riders?

> How to increase the number of annual memberships? 

> Is there any potential to convert casual user to annual memberships user?

# Dataset
I will use Cyclistic’s historical trip data to analyze and identify trends from [the previous 12 months of Cyclistic trip data](http://divvy-tripdata.s3.amazonaws.com/index.html). (Note: The datasets have a different name because Cyclistic is a fictional company. The data has been made available by Motivate International Inc. under this [license](http://ride.divvybikes.com/data-license-agreement).) This is public data that can be use to explore how different customer types are using Cyclistic bikes. But note that data-privacy issues is prohibited from using riders’ personally identifiable information. This means that we won’t be able to connect pass purchases to credit card numbers to determine if casual riders live in the
Cyclistic service area or if they have purchased multiple single passes.

The historical trip data is divided by month, each dataset has 13 columns representing user id, user type, rideable type, start & ending information for timestamp, station name, station id, and station location (latitude & longitude). 

# Prepare
After looked at the dataset, I decided to use the 12 months of historical data from March 2021 to Febuary 2022. I found that the accumulate row of these usage data is over millions, so I decided to use SQL (Google BigQuery) to clean and maniplate initial data. Then, I would use Tableau to represent geological data and R for other visualizations. 

# Processing data
In this section, I will describe the process and code for cleaning this dataset. Note that these SQL code is the example of what I used to clean data and it is not runnable in this notebook.

I started from uploading dataset to Google BigQuery and union each month's data using a code below

Note that some month data sizes are bigger than my BigQuery sandbox limitation, so I had to split my data in Excel before import them.

```sql
SELECT * FROM `202103-tripdata`
UNION DISTINCT
SELECT * FROM `202104-tripdata`
UNION DISTINCT
SELECT * FROM `202105-tripdata`
UNION DISTINCT
SELECT * FROM `202106-tripdata-1`
UNION DISTINCT
SELECT * FROM `202106-tripdata-2`
UNION DISTINCT
SELECT * FROM `202107-tripdata-1`
UNION DISTINCT
SELECT * FROM `202107-tripdata-2`
UNION DISTINCT
SELECT * FROM `202108-tripdata-1`
UNION DISTINCT
SELECT * FROM `202108-tripdata-2`
UNION DISTINCT
SELECT * FROM `202109-tripdata-1`
UNION DISTINCT
SELECT * FROM `202109-tripdata-2`
UNION DISTINCT
SELECT * FROM `202110-tripdata-1`
UNION DISTINCT
SELECT * FROM `202110-tripdata-2`
UNION DISTINCT
SELECT * FROM `202111-tripdata`
UNION DISTINCT
SELECT * FROM `202112-tripdata`
UNION DISTINCT
SELECT * FROM `202201-tripdata`
UNION DISTINCT
SELECT * FROM `202202-tripdata`
```

I saved this table as `tripdata_year_raw`

## Cleaning data

From the table the ride id column is the primary key, so I observed its characteristics by `LENGHT` function.

I found that most of the ride id has 16 char character length.

Therefore, I clean data by remove duplicate using `DISTINCT` function and filter out some invalid ride id by remove null value and id length (limit to only 16) using code below.

```sql 
SELECT DISTINCT *
FROM `tripdata_year_raw`
WHERE ride_id IS NOT NULL AND LENGTH(ride_id) = 16
```

Then, save this table as `tripdata_year_no_duplicate`

## Manipulating data
### Data for R

From the `tripdata_year_no_duplicate`, I want to extract data for analysis including 
* ride id (primary key)
* member type
* rideable type
* started timestamp
* ended timestamp
* year and month*
* day of week*
* time of the day*
* ride length* (time period)
* started station name
* ended station

Some of the information can be copied immediately from the previous table, however, information that have `*` mask have to obtain by manipulating some information from the original table.

Firstly, for year and month column, I used the original information from started timestamp column by using `EXTRACT` function to extract year and month from timestamp information. Then I use `CAST` function to cast the information to `STRING` type and then join them with `CONCAT` function.

Then, use `CASE` function to extract day of week and time of day information from the starting time column.

After that, calculate ride period using the ending time subtract by starting time.

In the end, avoid any invalid ride period value by filter only positive values.

The above process can be done using a code below

```sql
WITH decided_column AS
(
  SELECT 
      ride_id,
      rideable_type,
      member_casual,
      started_at,
      ended_at,
      CONCAT(CAST(EXTRACT(YEAR FROM started_at) AS STRING), "-", CAST(EXTRACT(MONTH FROM started_at) AS STRING)) AS year_month,
      CASE
          WHEN EXTRACT(DAYOFWEEK FROM started_at) = 1 THEN "Sunday"
          WHEN EXTRACT(DAYOFWEEK FROM started_at) = 2 THEN "Monday"
          WHEN EXTRACT(DAYOFWEEK FROM started_at) = 3 THEN "Tuesday"
          WHEN EXTRACT(DAYOFWEEK FROM started_at) = 4 THEN "Wednesday"
          WHEN EXTRACT(DAYOFWEEK FROM started_at) = 5 THEN "Thursday"
          WHEN EXTRACT(DAYOFWEEK FROM started_at) = 6 THEN "Friday"
          ELSE "Saturday"
      END AS day_of_week,
      CASE
          WHEN EXTRACT(HOUR FROM started_at) >= 6 AND EXTRACT(HOUR FROM started_at) < 11 THEN "morning"
          WHEN EXTRACT(HOUR FROM started_at) >= 11 AND EXTRACT(HOUR FROM started_at) < 16 THEN "afternoon"
          WHEN EXTRACT(HOUR FROM started_at) >= 16 AND EXTRACT(HOUR FROM started_at) < 21 THEN "evening"
          ELSE "night"
      END AS time_of_day,
      ended_at - started_at AS ride_length,
      start_station_name,
      end_station_name
  FROM `tripdata_year_no_duplicate` 
)
SELECT *
FROM decided_column
WHERE 
    ride_length > MAKE_INTERVAL(0, 0, 0, 0, 0, 0) 
    AND ride_length IS NOT NULL
```

I saved this result to `tripdata_year_cleaned`, which will be import later for analysis in R environment.

### Data for Tableau


Based on `tripdata_year_cleaned` table, I want to extract data for creating traffic dash board in Tableau. 

Since there is no latitude and longitude information for each station, I decided to determine latitude and longitude value by their average value first.

This can be done by retrieve all station name, with their latitude and longitude from `tripdata_year_no_duplicate` table in started station and ended station column then `UNION` them together and store it in temporary table called `location`. After that, `GROUP BY` the station name and average their latitude and longitude values.    

The above process can be explain using the code below

```sql
WITH location AS
(
  SELECT
    start_station_name AS station_name,
    start_lat AS lat,
    start_lng AS lng
  FROM `tripdata_year_no_duplicate`
  WHERE 
    start_station_name IS NOT NULL 
    AND start_lat IS NOT NULL 
    AND start_lng IS NOT NULL

  UNION ALL

  SELECT
    end_station_name AS station_name,
    end_lat AS lat,
    end_lng AS lng
  FROM `tripdata_year_no_duplicate`
  WHERE 
    end_station_name IS NOT NULL 
    AND end_lat IS NOT NULL 
    AND end_lng IS NOT NULL
)
SELECT
  station_name,
  AVG(lat) AS latitude,
  AVG(lng) AS longitude
FROM location
GROUP BY station_name
```

I saved this location information as `station_location` table.

After that, I join them with the `tripdata_year_cleaned` table with selected column are
* ride id
* rideable type
* member type
* year and month
* day of week
* time of the day
* ride length
* start station name
* start station latitude
* start station longitude
* end station name
* end station latitude
* end station longitude

by using code below

```sql
SELECT 
  tripdata.ride_id,
  tripdata.rideable_type,
  tripdata.member_casual,
  tripdata.year_month,
  tripdata.day_of_week,
  tripdata.time_of_day,
  tripdata.ride_length,
  tripdata.start_station_name,
  start_loc.latitude AS start_latitude,
  start_loc.longitude AS start_longitude,
  tripdata.end_station_name,
  end_loc.latitude AS end_latitude,
  end_loc.longitude AS end_longitude
FROM `cyclistic_final.tripdata_year_cleaned` AS tripdata
INNER JOIN `cyclistic_final.station_location` AS start_loc
ON tripdata.start_station_name = start_loc.station_name
INNER JOIN `cyclistic_final.station_location` AS end_loc
ON tripdata.end_station_name = end_loc.station_name
```

I saved this table as `tripdata_year_tableau`, then I exported it to create traffic dashboard using Tableau.

# Analyze
I used 2 different tools to analyze data in this project, R for tabular and chart visualizations, and Tableau for geological representation.

## R
I start loading data into R environment using code below

In [ ]:
# load library
library(tidyverse)
library(lubridate)
library(zoo)

In [ ]:
# set the working directory 

setwd("/kaggle/input/cyclistic-tripdata")

In [ ]:
# read data
df_year <- read.csv("tripdata_year_cleaned.csv")

In [ ]:
# view data
head(df_year)

In [ ]:
# check maximum ride length
max(df_year$ride_length)

In [ ]:
# convert data type and rename column
df_year <- df_year %>%
    mutate(
        started_at = ymd_hms(started_at),
        ended_at = ymd_hms(ended_at),
        year_month = as.factor(as.yearmon(year_month)),
        ride_length = str_remove(ride_length, "0-0 0 "),
        ride_length = hms(ride_length)) %>%
    rename(
        ride_type = rideable_type,
        member_type = member_casual,
        start_time = started_at,
        end_time = ended_at,
        start_station = start_station_name,
        end_station = end_station_name)

# view new data
head(df_year)

### Table

Next, I start to analyze data by creating more specific table.

Firstly, I count the total number of ride classify by member.

In [ ]:
df_year %>%
    count(member_type)

From above result, we can see that member user have a little bit more rides than casual users. 

Then, I want to find the top longest ride.

In [ ]:
# select top 200 longest ride by member type
df_longest <- df_year %>%
    select(c(member_type, ride_length)) %>%
    arrange(desc(ride_length)) %>%
    top_n(200, ride_length)

# view data
head(df_longest)

In [ ]:
# count number of user in top 200 longest ride by member
count(df_longest, member_type)

From the above result, we can see that there are only casual users in the top 200 longest ride. So I have an assumption that casual users are more likely to take long rides than member users.

To confirm that, let's find the average ride length for each user type.

In [ ]:
# calculate average ride length by member
df_year %>%
    mutate(ride_length = period_to_seconds(ride_length)/60) %>%
    group_by(member_type) %>%
    summarise("Average Ride Length (minute)" = mean(ride_length))

Therefore, we can conclude that casual users are more likely to take long rides than member users.

Next, I will find the most usage for start and end station.

In [ ]:
# retrieve station information from df_year
df_station <- df_year %>%
    select(member_type, start_station, end_station) %>%
    mutate_all(na_if,"") %>%
    drop_na()

head(df_station)

In [ ]:
# find the top 10 start station
df_station %>%
    count(start_station) %>%
    arrange(desc(n)) %>%
    head(n=10)

In [ ]:
# find the top 10 start station classify by member type
df_station %>%
    count(start_station, member_type) %>%
    group_by(start_station) %>%
    arrange(desc(n)) %>%
    head(n=10)

# this can be further filter by member type using code below
# filter(member_type=='member') %>%

From the above result, we can now know the most usage station for the start station and even can filter them by member. In the same way, we can also find these results for end station case (omitted here).

### Visualization

Next, I will use ggplot2 to analyze and visualize our dataset.

Firstly, find the number of user for each month

In [ ]:
# set working directory
setwd('/kaggle/working')

In [ ]:
# remove e+
options(scipen=999)

In [ ]:
# figure size function
fig <- function(width, heigth){
 options(repr.plot.width = width, repr.plot.height = heigth)
 }

In [ ]:
# overall usage by month
fig(8, 8)
ggplot(data = df_year) +
  geom_bar(mapping = aes(x = year_month), fill="steelblue")+
  labs(title="Number of User by Month", x = "Month - Year", y = "Number of User")+
  theme(text = element_text(size = 16), axis.text.x = element_text(angle = 90, vjust=0.6))

From the graph, we can see that the number of user rise in June to September period. Now, let's see what type of user effect this trend.

In [ ]:
# overall usage by month classify by member type
fig(12, 6)
ggplot(data = df_year) +
  geom_bar(mapping = aes(x = year_month), fill="steelblue")+
  facet_wrap(vars(member_type))+
  labs(title="Number of User by Month", x = "Month - Year", y = "Number of User")+
  theme(text = element_text(size = 16), axis.text.x = element_text(angle = 90, vjust=0.6))

We can see that when compare to member user the casual user have more drastic change in each month.

Then, I find the percentage of member type in each month.

In [ ]:
# percentage of user type in each month
fig(8, 8)
df_year %>%
    count(year_month, member_type) %>%
    group_by(member_type) %>%
    mutate(pct= prop.table(n) * 100) %>%
    ggplot() +
      geom_bar(mapping = aes(x = year_month, y = pct, fill = member_type), position="fill", stat="identity")+
      scale_fill_brewer(palette="Paired") +
      labs(title="Percentage of User by Month", x = "Month - Year", y = "Percentage of User", fill = "Member Type")+
      theme(text = element_text(size = 16), axis.text.x = element_text(angle = 90, vjust=0.6))



From the graph, we can see that member user have more percentage than casual user overall. However, casual user have the lead in peak period (June to September).

Next, I find ride types that preferred by each member type.

In [ ]:
# ride type preferred by each member type
fig(6, 6)
ggplot(data = df_year) +
  geom_bar(mapping = aes(x = ride_type), fill="steelblue")+
  facet_wrap(vars(member_type))+
  labs(title="Number of User by Ride Type", x = "Ride Type", y = "Number of User")+
  theme(text = element_text(size = 16), axis.text.x = element_text(angle = 90, vjust=0.6))

From above charts, casual user mainly use classic and electric bikes, with small amount of docked bike. On the other hand, member user prefer to use classic bike to electric bike, and did not use docked bike.

After that, I find the distribution of user over day of week.

In [ ]:
# day of week preferred by each member type
fig(8, 6)
ggplot(data = df_year) +
  geom_bar(mapping = aes(x = factor(day_of_week, level=c("Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"))), fill="steelblue")+
  facet_wrap(vars(member_type))+
  labs(title="Number of User by Day of Week", x = "Day of Week", y = "Number of User")+
  theme(text = element_text(size = 16), axis.text.x = element_text(angle = 90, vjust=0.6))

As stated in above graph, casual user tend to have more rides in the weekend, while member user have more rides in the weekday.

Furthermore, inserting the time distribution in each day.

In [ ]:
# day of week and time of day preferred by each member type
fig(8, 6)
ggplot(data = df_year) +
  geom_bar(mapping = aes(x = factor(day_of_week, level=c("Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday")), 
                        fill = factor(time_of_day, level=c("morning", "afternoon", "evening", "night"))))+
  scale_fill_brewer(palette="Paired") +
  facet_wrap(vars(member_type))+
  labs(title="Number of User by Day of Week and Time of Day", x = "Day of Week", y = "Number of User", fill = "Time of day")+
  theme(text = element_text(size = 16), axis.text.x = element_text(angle = 90, vjust=0.6))

From above chart, we can see that casual user prefer to have rides in weekend afternoon, while member user usually have rides on the evening of weekday.

Lastly, I find the user trend on their ride length.

In [ ]:
# ride length binning by user type
fig(8, 8)
df_year %>%
    mutate(ride_length_bin = cut(period_to_seconds(ride_length), 
                                 breaks=c(hms("0:0:0"), hms("0:15:0"), hms("0:30:0"), hms("2:0:0"), hms("1000:0:0")),
                                 labels=c("less than 15 m", "15 m to 30 m", "30 m to 2 hr", "more than 2 hr"))) %>%
    ggplot() +
      geom_bar(mapping = aes(x = factor(ride_length_bin, level=c("less than 15 m", "15 m to 30 m", "30 m to 2 hr", "more than 2 hr"))), fill="steelblue")+
      facet_wrap(vars(member_type))+
      labs(title="Number of User by Ride Length", x = "Ride Length", y = "Number of User")+
      theme(text = element_text(size = 16), axis.text.x = element_text(angle = 90, vjust=0.6))

From the chart, we can see that all user trend to prefer short rides. The casual user mainly have rides that less than 2 hours, while member user prefer shorter rides within 15 minutes.

## Tableau

After analyzing data in R, I use my dataset for Tableau, which I created earlier, and create user traffic map using dashboard.

In my dashboard, there are two different map indicate the distribution of user by start station and end station. The data in maps can be filter using member type, rideable type, year-month, day of week, and time of day.

You can visit my interactive dashboard with [this Tableau link](https://public.tableau.com/app/profile/supat.roongpraiwan/viz/CyclisticUserDistributionDashboard/CyclisticsUserDistributionDashboard).

Next, I will show the example of my dashboard output when applying no data filter along with their data retrieved from earlier analysis in R.

![all-user](https://github.com/supat-roong/Google-Data-Analytics-Capstone-Project/blob/main/cyclistic_all_user.png?raw=true)

From above dashboard, we can see that there is a huge cluster with a lot of rides in the area along the shore. Furthermore, there is one particular station that have number of rides outstanding from others (much darker color).

In [ ]:
# top 5 start station for all user
df_station %>%
    count(start_station) %>%
    arrange(desc(n)) %>%
    head(n=5)

In [ ]:
# top 5 end station for all user
df_station %>%
    count(end_station) %>%
    arrange(desc(n)) %>%
    head(n=5)

From the table data, we now know that the outstanding point in the map is Streeter Dr & Grand Ave station with around 80,000 rides.

Now, lets filter this dashboard by member type to observe each member type behavior, starting with casual user.

![casual](https://github.com/supat-roong/Google-Data-Analytics-Capstone-Project/blob/main/cyclistic_casual.png?raw=true)

From the above map, the casual user have alot of usage mostly in one outstanding spot and their neighbourhood.

In [ ]:
# top 5 start station for casual user
df_station %>%
    filter(member_type=='casual') %>%
    count(start_station, member_type) %>%
    group_by(start_station) %>%
    arrange(desc(n)) %>%
    head(n=5)

In [ ]:
# top 5 end station for casual user
df_station %>%
    filter(member_type=='casual') %>%
    count(end_station, member_type) %>%
    group_by(end_station) %>%
    arrange(desc(n)) %>%
    head(n=5)

From these casual user rides data, the most popular station is Streeter Dr & Grand Ave station, which is matched with the previous overall result.

Next, for the membership user

![member](https://github.com/supat-roong/Google-Data-Analytics-Capstone-Project/blob/main/cyclistic_member.png?raw=true)

As in above map, unlike casual user, member user station usage distribution are quite vary with a huge cluster along the shore area. This is match withed the cluster in the overall result.

In [ ]:
# top 5 start station for member user
df_station %>%
    filter(member_type=='member') %>%
    count(start_station, member_type) %>%
    group_by(start_station) %>%
    arrange(desc(n)) %>%
    head(n=5)

In [ ]:
# top 5 end station for member user
df_station %>%
    filter(member_type=='member') %>%
    count(end_station, member_type) %>%
    group_by(end_station) %>%
    arrange(desc(n)) %>%
    head(n=5)

From these member user data, although the most usage station is Clark St & Elm St station, there is not much difference between the top usage station camparing to the casual user case.

# Conclusion

From above analysis, we noticed some usage trend summarizeed in the key takeaways below.
* Usage tend to rise in June to September period, this migth be affect by the weather condition because those months are in summer period.
* Overall, the number of user went up as the rides are shorter.
* From user distribution map, there were a lot of rides in the area along the shore with the peak usage at Streeter Dr & Grand Ave station.

Next, I noted the difference of member and casual user behavior in below.
* Change in each month
    * Member user have more consistency
    * Casual user have more drastic change
* Ride types
    * Member user prefer to use classic bike than electric bike, and not using docked bike
    * Casual user use all three types of bike, but mainly prefer classic bike
* Time and day of rides
    * Member user usually have rides in weekday evening
    * Casual user prefer to have rides in weekend afternoon
* Ride length
    * Average trip duration of casual users is more than twice that of member users
    * Member user prefer shorter rides within 15 minutes
    * Casual user mainly have rides that less than 2 hours
* Location distribution
    * Member user location distribution are quite vary with a huge cluster along the shore area
    * Casual user have drastic usage mostly in one particular spot at Streeter Dr & Grand Ave station
    
Despite of the differences between member and casual user, there are some segments in casual users that overlap with member user. Therefore, I suggest to increase annual membership user by using below method.
* Offer discounted price for new member registration
* Promote the membership campaign along shore area.

On the other hand, we should also attracts new casual user because their number of rides are as much as the member user and they spend more time in each rides. These are my recommendations below.
* Offer weekend pass and 2-hour-ride package deals
* Promote casual ride campaign around Streeter Dr & Grand Ave station

Lastly, we can improve bike allocation efficency with the time and location usage data.
* Provide more bikes for member user around shore area in the weekday
* Provide more bikes for casual user in weekend afternoon at Streeter Dr & Grand Ave station area

That is the end of my report, thank you for your reading.